In [ ]:
#python libraries
import os
import re
import copy

#other ML libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#ploting
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Setting random seeds for reproducibility
random_seed = 420
np.random.seed(random_seed)
_ = torch.manual_seed(random_seed)

### DATA CLEANING

In [ ]:
#Import Disruptive & Netatmo Sensor Data

# assign variables
directory = './storedData/'
sensor_data = pd.DataFrame()
sensor_data['timestamp'] = None #in order to merge based on it 

measurement_types = ["Door", "Water", "Humidity_temperature", "Humidity_humidity", 
                     "Humidifier_humidity", "Humidifier_temperature","Temperature",
                     "Hum_temperature", "Hum_humidity"]
discrete_measurement_types = ["Door", "Water"]


# iterate over files in that directory and concat into 1 big dataframe based on timestamps
for filename in os.listdir(directory): 
    file = os.path.join(directory, filename)
    if os.path.isfile(file):    
        df = pd.read_csv(file)
        df.drop(columns=df.columns[0], inplace=True) # dropping unnamed coloumn for indexing in .cvs file

        df['timestamp'] = pd.to_datetime(df['timestamp'],  utc=True)#.dt.tz_convert('Europe/Rome') #set timezone correctly if not correct in local time
        df.set_index("timestamp", inplace=True)
        for measurement_name in df.columns:
            measurement_frame = df[measurement_name].to_frame()

            measurement_name_list = re.findall("([A-Z0-9][^A-Z]*)", measurement_name) # Splitting string on capital letters
            print(f'filename: {filename}, measurement_name_list: {measurement_name_list}')
            measurement_type = measurement_name_list[-1]
            if measurement_type not in measurement_types: #Temperature measurements does not have temperature in their name
                measurement_type = "Temperature"
                measurement_name_list.append(measurement_type)
            #Renaming coloumn for consistent naming
            coloumn_name = f"{''.join(measurement_name_list[:-1])}_{measurement_type.lower()}"
            measurement_frame.rename(columns={measurement_frame.columns[0]: coloumn_name}, inplace=True)
            # print(f"key: {measurement_frame.columns[0]} Value: {coloumn_name}")
            if measurement_type in discrete_measurement_types:
                #For Water Detection:
                #NOT_PRESENT == No water detected = 0
                #PRESENT     == water detected    = 1

                #For Door Sensors: - this is because sensor changes state based on if it detects objects in front of it
                #NOT_PRESENT == DOOR IS OPEN     = 0
                #PRESENT     == DOOR IS CLOSED   = 1
                
                measurement_frame.replace(['NOT_PRESENT', 'PRESENT'],[0, 1], inplace=True)
                measurement_frame = measurement_frame.resample('15T').ffill()
            else: #Contious measurement
                measurement_frame  = measurement_frame.resample('15T').mean(numeric_only=True).interpolate() #Resample to managable timestamps by taking mean of 5 minute slots and linear interpolating NaNs, i.e. 5, 10, 15, 20 minutes etc..

    
            sensor_data = pd.merge(sensor_data, measurement_frame, on='timestamp', how="outer")  

sensor_data.set_index("timestamp", inplace=True)
sensor_data.sort_index(axis=0, inplace=True) #Sorting by timestamp
sensor_data.sort_index(axis=1, inplace=True) #Sorting columns alpabetically

#Postprocesing the data in order to get rid of the NaN Values
#ffill and fillna(0/1) to populate NaN values of the proximity with ffill and then set the values before first valid value to DOOR_CLOSED/NO_WATER
sensor_data.loc[:, sensor_data.columns.str.endswith('door')] = sensor_data.loc[:, sensor_data.columns.str.endswith('door')].ffill().fillna(1)
sensor_data.loc[:, sensor_data.columns.str.endswith('water')] = sensor_data.loc[:, sensor_data.columns.str.endswith('water')].ffill().fillna(0)


# # #linearly interpolate temperature and humidity values:
sensor_data.loc[:, sensor_data.columns.str.endswith('temperature')]  = sensor_data.loc[:, sensor_data.columns.str.endswith('temperature')].interpolate().bfill().ffill()
sensor_data.loc[:, sensor_data.columns.str.endswith('humidity')]  = sensor_data.loc[:, sensor_data.columns.str.endswith('humidity')].interpolate().bfill().ffill()

### EXPLORATORY DATA ANALYSIS ###

In [ ]:
#plot the data
sensor_data.plot(figsize=(16, 16))

### FEATURE ENGINEERING ###

In [ ]:
#Add hand-designed time-of-day and day-of-year features
min_in_hour = 60
min_in_day = 24*min_in_hour
sensor_data['sin_daily'] = np.sin(2*np.pi*(sensor_data.index.minute + sensor_data.index.hour*min_in_hour)/min_in_day).values
sensor_data['cos_daily'] = np.cos(2*np.pi*(sensor_data.index.minute + sensor_data.index.hour*min_in_hour)/min_in_day).values

days_in_year = 365.2425
sensor_data['sin_yearly'] = np.sin(2*np.pi*(sensor_data.index.day_of_year)/days_in_year).values
sensor_data['cos_yearly'] = np.cos(2*np.pi*(sensor_data.index.day_of_year)/days_in_year).values

days_in_week = 7
sensor_data['sin_weekly'] = np.sin(2*np.pi*(sensor_data.index.day_of_week)/days_in_week).values
sensor_data['cos_weekly'] = np.cos(2*np.pi*(sensor_data.index.day_of_week)/days_in_week).values

In [ ]:
sensor_data.describe()

### DATA PREPROCESSING

In [ ]:
train_val_df = sensor_data.iloc[:int(sensor_data.shape[0]*0.9)]
test_df = sensor_data.iloc[int(sensor_data.shape[0]*0.9):]

train_val_split = int(train_val_df.shape[0]*0.8)
train_df = train_val_df[:train_val_split]
val_df = train_val_df[train_val_split:]

train_labels = train_df.loc[:, sensor_data.columns.str.endswith('temperature')].drop("1OutdoorEntrance_temperature", axis=1).sort_index(axis=1)

x_scaler = StandardScaler()
y_scaler = MinMaxScaler()

x_scaler.fit(train_df)
y_scaler.fit(train_labels)

# train_df_norm = scaler.fit_transform(train_df)
# val_df_norm = scaler.transform(val_df)
# test_df_norm = scaler.transform(test_df)


print(f"Shape of train_df = {train_df.shape}, Shape of val_df = {val_df.shape}, Shape of test_df = {test_df.shape}")
print(f"Shape of train_labels = {train_labels.shape}")

In [ ]:
import WindowGenerator
import importlib
importlib.reload(WindowGenerator)

windows = WindowGenerator.WindowGenerator(4*24, 1, 0, train_df, val_df, test_df, train_labels.columns)

In [ ]:
# x_train = np.array(windows.train_data, dtype=object)[:,0]
# y_train = np.array(windows.train_data, dtype=object)[:,1]

# x_val = np.array(windows.val_data, dtype=object)[:,0]
# y_val = np.array(windows.val_data, dtype=object)[:,1]

# x_test = np.array(windows.test_data, dtype=object)[:,0]
# y_test = np.array(windows.test_data, dtype=object)[:,1]

In [ ]:
x_train_norm = []
y_train_norm = []
for inputs, labels in windows.train_data:
    x_train_norm.append(x_scaler.transform(inputs))
    y_train_norm.append(y_scaler.transform(labels))

x_train_norm = np.array(x_train_norm)
x_train_norm = torch.from_numpy(x_train_norm).float()
y_train_norm = np.array(y_train_norm)
y_train_norm = torch.from_numpy(y_train_norm.reshape(y_train_norm.shape[0], y_train_norm.shape[2])).float()

In [ ]:
y_test_norm = []
y_val_norm = []
for inputs, labels in windows.val_data:
    y_test_norm.append(x_scaler.transform(inputs))
    y_val_norm.append(y_scaler.transform(labels))

y_test_norm = np.array(y_test_norm)
y_test_norm = torch.from_numpy(y_test_norm).float()
y_val_norm = np.array(y_val_norm)
y_val_norm = torch.from_numpy(y_val_norm.reshape(y_val_norm.shape[0], y_val_norm.shape[2])).float()

In [ ]:
x_test_norm = []
y_test_norm = []
for inputs, labels in windows.test_data:
    x_test_norm.append(x_scaler.transform(inputs))
    y_test_norm.append(y_scaler.transform(labels))

x_test_norm = np.array(x_test_norm)
x_test_norm = torch.from_numpy(x_test_norm).float()
y_test_norm = np.array(y_test_norm)
y_test_norm = torch.from_numpy(y_test_norm.reshape(y_test_norm.shape[0], y_test_norm.shape[2])).float()

### LSTM

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_lstm_layers, linear_layers=[]):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_lstm_layers = num_lstm_layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_lstm_layers, batch_first=True)
        if linear_layers == []:
            self.linear_layers = [nn.Linear(hidden_size, output_size)]
        else:
            #He initialization
            first_layer = nn.Linear(hidden_size, linear_layers[0])
            first_layer.weight.data = torch.randn((hidden_size, linear_layers[0]))*np.sqrt(1/hidden_size)
            first_layer.bias.data = torch.zeros(linear_layers[0])
            self.linear_layers = [first_layer]

            for i in range(len(linear_layers)-1):
                n_in = linear_layers[i]
                n_out = linear_layers[i+1]
                layer = torch.nn.Linear(n_in, n_out)

                layer.weight.data = torch.randn((n_out, n_in))*np.sqrt(2/n_in)
                layer.bias.data = torch.zeros(n_out)
                self.linear_layers.append(layer)
            
            last_layer = nn.Linear(linear_layers[-1], output_size)
            last_layer.weight.data = torch.randn((n_out, n_in))*np.sqrt(2/n_in)
            last_layer.bias.data = torch.zeros(n_out)
            self.linear_layers.append(last_layer)

        self.linear_layers = nn.ModuleList(self.linear_layers)
        self.act =nn.ReLU()
        

    def forward(self, input_seq):
        torch.autograd.set_detect_anomaly(True)
        h_0 = Variable(torch.zeros(self.num_lstm_layers, input_seq.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_lstm_layers, input_seq.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        # output, (hn, cn) = self.lstm(input_seq, (h_0, c_0)) #lstm with input, hidden, and internal state
        # x = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        output, (hn, cn) = self.lstm(input_seq, (h_0, c_0))
        x = hn.view(-1, self.hidden_size)
        for layer in self.linear_layers:
            x = self.act(x)
            x = layer(x)
        return x

In [ ]:
def train(
        net: torch.nn.Module,
        train_data: list,
        val_data: list,
        n_epochs: int,
        lr: float,
        l2_reg: float,
) -> torch.nn.Module:
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr = lr)

    for epoch in range(n_epochs):
        # for inputs, targets in train_data:
        optimizer.zero_grad()
        # net.hidden_cell = (torch.zeros(1, 1, net.hidden_size),
        #                         torch.zeros(1, 1, net.hidden_size))

        outputs = net(train_data[0])
        batch_mse = criterion(outputs, train_data[1])
        reg_loss = 0
        for param in net.parameters():
                reg_loss += param.abs().sum()
        cost = batch_mse + l2_reg * reg_loss
        cost.backward()
        optimizer.step()

        # if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss: {cost.item()}')

        # mse_val = 0
        # if epoch % 10 == 0:
        #         for inputs, label in val_data:
        #                 pred = y_scaler.inverse_transform(net(inputs).detach().numpy())
        #                 row_len = len(label[0])
        #                 mse_val += np.sum(np.power(label-pred, 2))
        #         mse_val /= len(val_data)*row_len
        #         print(f'Epoch: {epoch+1}: Val MSE: {mse_val}')
    return net
    

In [ ]:
net = Net(input_size=35, hidden_size=128, output_size=17, num_lstm_layers=1)

epochs = 500
lr = 0.001
l2_reg = 0
train(net, [x_train_norm, y_train_norm], windows.test_data, epochs, lr, l2_reg)

### EXAMINE MODEL

In [ ]:
# Predict on validation data
pred_val_inv = y_scaler.inverse_transform(net(y_test_norm).detach())
# pred_val = torch.from_numpy(pred_val).to(torch.float)
y_val_inv = y_scaler.inverse_transform(y_val_norm)

# Compute MSE, MAE and MAPE on validation data
print('Error on validation data')

mse_val = np.mean(np.power((pred_val_inv - y_val_inv), 2))
print(f'MSE: {mse_val.item()}')

mae_val = np.mean(np.abs(pred_val_inv - y_val_inv))
print(f'MAE: {mae_val.item()}')

# mape_val = 100*np.mean(np.abs(np.divide(pred_val_inv - y_val_inv, y_val_inv)))
# print(f'MAPE: {mape_val.item()} %')

In [ ]:
net.eval()
pred_train = y_scaler.inverse_transform(net(y_test_norm).detach().numpy())
true_train = y_scaler.inverse_transform(y_val_norm.detach().numpy())
net.train()

fig,ax = plt.subplots(7,3,figsize=(15,25))
for i in range(train_labels.shape[1]):
    ax[i//3, i%3].plot(true_train[:, i], label="true")
    ax[i//3, i%3].plot(pred_train[:, i], '--', label="pred")
    ax[i//3, i%3].set_title(f"{train_labels.columns[i]}")
    ax[i//3, i%3].legend()

### PyTorch Forecasting library